In [10]:
from FaceLandmarkDetection.src.quantization.helper import *
import torch

In [11]:
random_seed = 0
num_classes = 136
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model_dir = "checkpoints"
model_filename = "resnet18_FLM.pt"
model_filepath = os.path.join(model_dir, model_filename)
set_random_seeds(random_seed=random_seed)

In [12]:
# Create an untrained model.
model = create_model(num_classes=num_classes)
train_dataset, val_dataset = get_data()
train_loader = make_loader(train_dataset, 64)
val_loader = make_loader(val_dataset, 32)

1111


In [13]:
model = load_model(model=model, model_filepath=model_filepath, device=cpu_device)

In [5]:
save_torchscript_model(model=model, model_dir="temp", model_filename="temp1")
ts_model = load_torchscript_model(model_filepath="temp/temp1", device=cpu_device)

In [7]:
evaluate_model(model=ts_model, test_loader=val_loader, device=cpu_device, criterion=nn.MSELoss())

0.023867265229021106

In [8]:
measure_inference_latency(model=ts_model, device=cpu_device, input_size=(32,1,224,224), num_samples=100)

1.6400473976135255

ONNX

In [1]:
import onnxruntime

In [2]:
onnx_model_dir = "checkpoints_onnx"
onnx_model_filename = "resnet.onnx"
onnx_model_filepath = os.path.join(onnx_model_dir, onnx_model_filename)

In [3]:
def evaluate_onnx_model(onnx_session, device, data_loader, criterion=None):
    running_loss = 0
    for inputs, labels in data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.view(labels.size(0), -1).float()
        ort_inputs = {onnx_session.get_inputs()[0].name: to_numpy(inputs)}
        ort_outs = onnx_session.run(None, ort_inputs)
        loss = criterion(torch.tensor(ort_outs[0]), labels).item()
        # statistics
        running_loss += loss * inputs.size(0)
        
    eval_loss = running_loss / len(data_loader.dataset)
    return eval_loss
        
def measure_inference_latency_onnx(onnx_session, device, input_size=(32,1,224,224), num_samples=100):
    start_time = time.time()
    for _ in range(num_samples):
        dummy_input = torch.randn(input_size, device=device)
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
        ort_outs = ort_session.run(None, ort_inputs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples
    return elapsed_time_ave 

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [1]:
dummy_input = torch.randn(32, 1, 224, 224, device='cpu')
input_names = ['input'],   # the model's input names
output_names = ['output'], # the model's output names

# Export the model
torch.onnx.export(model,               # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  onnx_model_filepath,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

In [4]:
ort_session = onnxruntime.InferenceSession(onnx_model_filepath)

In [8]:
# compute ONNX Runtime output prediction
dummy_input = torch.randn(1, 1, 224, 224, device='cpu')
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

In [14]:
evaluate_onnx_model(onnx_session=ort_session, data_loader=val_loader, device=cpu_device, criterion=nn.MSELoss())

0.024484903627150768

In [15]:
measure_inference_latency_onnx(ort_session, "cpu", input_size=(32,1,224,224), num_samples=100)

0.2827521514892578